In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
from math import sqrt
import seaborn as sns
import random

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

from functions_and_objects import (impute_means_zeros_maxs_X, parse_percentage,
                                   clean_LC_data_classification_eval, preprocessing_eval,
                                   nan_max_cols, nan_mean_cols, nan_zero_cols, dtype,
                                   one_hot_encode_eval, concat_X_and_6ohe_dfs,
                                   prep_df_for_regression_eval,
                                   regression_model_eval_prep_pipeline)

import joblib

In [5]:
raw_q42017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q4.csv',header=1)
raw_q32017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q3.csv',header=1)
raw_q22017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q2.csv',header=1)
raw_q12017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q1.csv',header=1)
raw_q42016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q4.csv',header=1)
raw_q32016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q3.csv',header=1)
raw_q22016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q2.csv',header=1)
raw_q12016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q1.csv',header=1)
raw_2015 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2015.csv',header=1)
raw_2014 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2014.csv',header=1)
raw_2012to2013 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2012to2013.csv',header=1)
raw_2007to2011 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2007to2011.csv',header=1)

In [6]:
dfs_list = [raw_q42017,raw_q32017,raw_q22017,raw_q12017,
           raw_q42016,raw_q32016,raw_q22016,raw_q12016,raw_2015,
           raw_2014,raw_2012to2013, raw_2007to2011]

In [7]:
(X_train_regr, X_test_regr, y_train_regr, y_test_regr) = regression_model_eval_prep_pipeline(dfs_list)

/home/jupyter/peervest/functions_and_objects.py:378: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  raw_lc_df['earliest_cr_line'] = pd.to_timedelta(pd.to_datetime(raw_lc_df['earliest_cr_line'])).dt.days
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/jupyter/peervest/functions_and_objects.py:280: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X_train[col] = X_train[col].fillna(0.0)
/home/jupyter/peervest/functions_and_objects.py:281: Setti

### Subsample DataFrames to quickly iterate on Random Forest Regressor

In [30]:
X_train_regr_samp = X_train_regr.sample(frac=.3,random_state=0)
X_test_regr_samp = X_test_regr.sample(frac=.3,random_state=0)
y_train_regr_samp = y_train_regr.sample(frac=.3,random_state=0)
y_test_regr_samp = y_test_regr.sample(frac=.3,random_state=0)

In [31]:
print(X_train_regr_samp.shape)
print(X_test_regr_samp.shape)
print(y_train_regr_samp.shape)
print(y_test_regr_samp.shape)

(264285, 1101)
(113265, 1101)
(264285, 1)
(113265, 1)


### Random Forest

In [39]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)

In [33]:
#rf.fit(X_train_regr_samp, np.array(y_train_regr_samp['annu_return']))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
                      max_features=900, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

##### GridSearch

In [40]:
param_grid = {
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2, 4],
    'max_depth': [10,15]
}

In [32]:
rfgs = GridSearchCV(rf, param_grid, cv=2)

In [33]:
rfgs.fit(X_train_regr_samp, np.array(y_train_regr_samp['annu_return']))

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=10, max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 15], 'min_samples_leaf': [1, 2, 4],
                         'min

In [34]:
rfgs.best_params_

{'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2}

In [34]:
y_train_regr_preds_samp = rfgs.predict(X_train_regr_samp)

In [35]:
y_test_regr_preds_samp = rfgs.predict(X_test_regr_samp)

### Random Forest v1 Evaluation
###### Params: n_estimators=30, min_samples_split=2, min_samples_leaf=2, max_depth=4

In [27]:
##BENCHMARK SET BY RIDGE REGRESSION V2:
#TEST ERROR METRICS:
#R-Squared: 0.6609090923805967
#Mean Squared Error: 0.021143184815253565
#Root Mean Squared Error: 0.14540696274681472

print("TRAIN ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_train_regr, y_train_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_train_regr, y_train_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_train_regr, y_train_regr_preds))))
print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_train_regr, y_train_regr, 
                                                                       scoring='neg_mean_squared_error')))

TRAIN ERROR METRICS:
R-Squared: 0.7337991886574766
Mean Squared Error: 0.016646060054940577
Root Mean Squared Error: 0.12901961112536567


In [28]:
print("TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr, y_test_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr, y_test_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr, y_test_regr_preds))))
print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_test_regr, y_test_regr, 
                                                                       scoring='neg_mean_squared_error')))

TEST ERROR METRICS:
R-Squared: 0.7327733043317483
Mean Squared Error: 0.016662267513303457
Root Mean Squared Error: 0.12908240590143746


In [29]:
# save model joblib
filename = 'rf_regr_v1.joblib'
joblib.dump(rf, filename)

['rf_regr_v1.joblib']

### Random Forest v2 Evaluation
###### 10% of values
###### Params: n_estimators=100, max_depth=10

In [17]:
##BENCHMARK SET BY RIDGE REGRESSION V2:
#TEST ERROR METRICS:
#R-Squared: 0.6609090923805967
#Mean Squared Error: 0.021143184815253565
#Root Mean Squared Error: 0.14540696274681472

print("TRAIN ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_train_regr_samp, y_train_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_train_regr_samp, y_train_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TRAIN ERROR METRICS:
R-Squared: 0.9750869957638799
Mean Squared Error: 0.0015561404203897633
Root Mean Squared Error: 0.039447945705572085


In [18]:
print("TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr_samp, y_test_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_test_regr_samp, y_test_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TEST ERROR METRICS:
R-Squared: 0.9723227392638374
Mean Squared Error: 0.001752411083397318
Root Mean Squared Error: 0.0418618093660238


In [19]:
# save model joblib
filename = 'rf_regr_v2.joblib'
joblib.dump(rf, filename)

['rf_regr_v2.joblib']

In [20]:
# TEST ON 100% of data
y_train_regr_preds = rf.predict(X_train_regr)
y_test_regr_preds = rf.predict(X_test_regr)

In [21]:
print("100% of DATA - TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr, y_test_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr, y_test_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr, y_test_regr_preds))))

100% of DATA - TEST ERROR METRICS:
R-Squared: 0.969368915436507
Mean Squared Error: 0.0019099264163830176
Root Mean Squared Error: 0.0437027049092275


### Random Forest v3 Evaluation
###### 20% of values
###### Params: n_estimators=100, max_depth=10

In [28]:
##BENCHMARK SET BY RIDGE REGRESSION V2:
#TEST ERROR METRICS:
#R-Squared: 0.6609090923805967
#Mean Squared Error: 0.021143184815253565
#Root Mean Squared Error: 0.14540696274681472

print("TRAIN ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_train_regr_samp, y_train_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_train_regr_samp, y_train_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TRAIN ERROR METRICS:
R-Squared: 0.9769980126707797
Mean Squared Error: 0.0014469458575561457
Root Mean Squared Error: 0.038038741534863446


In [29]:
print("TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr_samp, y_test_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_test_regr_samp, y_test_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TEST ERROR METRICS:
R-Squared: 0.973743977442095
Mean Squared Error: 0.0016606853024841335
Root Mean Squared Error: 0.040751506751089996


In [30]:
# save model joblib
filename = 'rf_regr_v3.joblib'
joblib.dump(rf, filename)

['rf_regr_v3.joblib']

In [31]:
# TEST ON 100% of data
y_train_regr_preds = rf.predict(X_train_regr)
y_test_regr_preds = rf.predict(X_test_regr)

In [32]:
print("100% of DATA - TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr, y_test_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr, y_test_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr, y_test_regr_preds))))

100% of DATA - TEST ERROR METRICS:
R-Squared: 0.9732666874699176
Mean Squared Error: 0.001666890367293121
Root Mean Squared Error: 0.040827568716409274


### Random Forest v4 Evaluation
###### DROP 'total_pymnt' column
###### 20% of values
###### Params: n_estimators=100, max_depth=10

In [13]:
##BENCHMARK SET BY RIDGE REGRESSION V2:
#TEST ERROR METRICS:
#R-Squared: 0.6609090923805967
#Mean Squared Error: 0.021143184815253565
#Root Mean Squared Error: 0.14540696274681472

print("TRAIN ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_train_regr_samp, y_train_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_train_regr_samp, y_train_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TRAIN ERROR METRICS:
R-Squared: 0.6184741107168366
Mean Squared Error: 0.02399998300787739
Root Mean Squared Error: 0.15491927900644706


In [14]:
print("TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr_samp, y_test_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_test_regr_samp, y_test_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TEST ERROR METRICS:
R-Squared: 0.5565794558387958
Mean Squared Error: 0.02804621221222714
Root Mean Squared Error: 0.16747003377388786


In [15]:
# save model joblib
filename = 'rf_regr_v4.joblib'
joblib.dump(rf, filename)

['rf_regr_v4.joblib']

In [16]:
# TEST ON 100% of data
y_train_regr_preds = rf.predict(X_train_regr)
y_test_regr_preds = rf.predict(X_test_regr)

In [17]:
print("100% of DATA - TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr, y_test_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr, y_test_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr, y_test_regr_preds))))

100% of DATA - TEST ERROR METRICS:
R-Squared: 0.5559341420805827
Mean Squared Error: 0.027688641285164094
Root Mean Squared Error: 0.16639904232045355


### Random Forest v5 Evaluation
###### DROP 'total_pymnt' column
###### 10% of values
###### Params: n_estimators=100, max_depth=10
###### APPLY GridSearchCV with cv=2

In [37]:
##BENCHMARK SET BY RIDGE REGRESSION V2:
#TEST ERROR METRICS:
#R-Squared: 0.6609090923805967
#Mean Squared Error: 0.021143184815253565
#Root Mean Squared Error: 0.14540696274681472

print("TRAIN ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_train_regr_samp, y_train_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_train_regr_samp, y_train_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_train_regr_samp, y_train_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TRAIN ERROR METRICS:
R-Squared: 0.7618539196965637
Mean Squared Error: 0.014875313230199153
Root Mean Squared Error: 0.12196439328836574


In [38]:
print("TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr_samp, y_test_regr_preds_samp)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr_samp, y_test_regr_preds_samp))))
#print("Cross Validated Mean Squared Error: {}".format(-cross_val_score(rf,X_test_regr_samp, y_test_regr_samp, 
#                                                                       scoring='neg_mean_squared_error')))

TEST ERROR METRICS:
R-Squared: 0.5666762542919535
Mean Squared Error: 0.027436289375481895
Root Mean Squared Error: 0.1656390333691968


In [39]:
# save model joblib
filename = 'rf_regr_v5.joblib'
joblib.dump(rfgs, filename)

['rf_regr_v5.joblib']

In [40]:
# TEST ON 100% of data
y_train_regr_preds = rfgs.predict(X_train_regr)
y_test_regr_preds = rfgs.predict(X_test_regr)

In [41]:
print("100% of DATA - TEST ERROR METRICS:")
print("R-Squared: {}".format(r2_score(y_test_regr, y_test_regr_preds)))
print("Mean Squared Error: {}".format(mean_squared_error(y_test_regr, y_test_regr_preds)))
print("Root Mean Squared Error: {}".format(sqrt(mean_squared_error(y_test_regr, y_test_regr_preds))))

100% of DATA - TEST ERROR METRICS:
R-Squared: 0.5556597318617378
Mean Squared Error: 0.02770575146370873
Root Mean Squared Error: 0.16645044747223942


In [42]:
rfgs.best_params_

{'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2}